# Using SDSS-V Targeting Flags with `semaphore`

## Learning Goals
By the end of this tutorial, you will:
- How to select spectra by their targeting information
- How to get the targeting information of a specific target

## Introduction
The SDSS-V Survey uses a complex targeting schema, where targets are selected through many cartons associated with the different SDSS-V mappers and programs. 

In this short tutorial, we will briefly cover determining which targets in a summary file are associated with a particular carton, program, or master. It also describes how to determine the targeting information for a single source. 

## Imports

This tutorial requires two commonly used python packages and two SDSS specific packages
- numpy for filtering of arrays
- astropy for reading fits tables
- sdss-access for accessing data on the SAS
- semaphore for exploring the flags

Missing packages can be installed via `pip install numpy astropy sdss-access sdss-semaphore`


NOTE: You may want to set the `$SAS_BASE_DIR` environment variable before starting this notebook, other wise `$HOME/sas` will be created and used



In [1]:
import os
os.environ['SDSSC2BV'] ='1' # to set the DR19 Carton to Bit Mapping Version

import numpy as np
from astropy.io import fits
from sdss_access import Access
from sdss_semaphore.targeting import TargetingFlags

# setup sdss_access for DR19
access = Access(release='DR19')

## Load the Data
We will use `sdss-access` to find/download the file. `sdss-access` has extensive [documentation](https://sdss-access.readthedocs.io/en/latest/), but we should mention a few things. First if you are a member of the collaboration accessing proprietary data, you need to set up a a [.netrc file](https://sdss-access.readthedocs.io/en/latest/auth.html). After that usage is the same for everyone.

You need specify the file species you need, in this case 'mwmStar', then a series of key word arguments that vary depending on the file species. A list of file species and their keyword arugments is [available](https://sdss-access.readthedocs.io/en/latest/paths.html).

The code below show an example of retreiving a summary file. access.full() builds the file path, access.exists() checks if the file exists, and access.commit() will retrieve the file and save it to your local SAS (`$SAS_BASE_DIR`) with the same structure as the SAS (so it will be many files deep).

In [2]:
filename = access.full('mwmAllVisit',v_astra = '0.6.0')
if not access.exists('',full=filename):
    # if the file does not exist locally, this code will download the data.
    access.remote()
    access.add('mwmAllVisit',v_astra = '0.6.0')
    access.set_stream()
    access.commit()
print(filename)

/Users/jdonor/sas/sdsswork/mwm/spectro/astra/0.6.0/summary/mwmAllVisit-0.6.0.fits.gz


### Open the file
Now use `astropy.fitsio` to load the data
- HDU 1 contains the BOSS Spectra
- HDU 2 contains the APOGEE Spectra

In [3]:
hdul = fits.open(filename)
hdul[0].header
boss_data = hdul[1].data

### Converting the stored column to a set of Targeting Flags
Now use the semaphore package to convert the fits column to Targeting Flags

In [4]:
flags = TargetingFlags(boss_data["SDSS5_TARGET_FLAGS"])

In [5]:
# Let's count the number of targets assigned to each carton (and skip empty cartons)
for label, count in flags.count(skip_empty=True).items():
    print(f"{label}: {count}")

mwm_snc_100pc_0.1.0: 33542
mwm_snc_250pc_0.1.0: 2019
mwm_cb_300pc_0.1.0: 123
mwm_cb_cvcandidates_0.1.0: 834
mwm_halo_sm_0.1.0: 1642
mwm_halo_bb_0.1.0: 31926
mwm_yso_s1_0.1.0: 9514
mwm_yso_s2_0.1.0: 44
mwm_yso_s2-5_0.1.0: 3
mwm_yso_s3_0.1.0: 15969
mwm_yso_ob_0.1.0: 16
mwm_yso_cluster_0.1.0: 16760
mwm_rv_long-fps_0.1.0: 2507
mwm_rv_long-bplates_0.1.0: 586
mwm_ob_cepheids_0.1.0: 128
mwm_rv_short-fps_0.1.0: 7162
mwm_rv_short-bplates_0.1.0: 847
mwm_ob_core_0.1.0: 101779
mwm_rv_short-rm_0.1.0: 153
mwm_rv_long-rm_0.1.0: 53
ops_std_boss_0.1.0: 132780
mwm_wd_core_0.1.0: 36240
mwm_gg_core_0.1.0: 5264
mwm_planet_tess_0.1.0: 4783
bhm_aqmes_med_0.1.0: 1691
mwm_cb_gaiagalex_0.1.0: 42430
bhm_aqmes_med-faint_0.1.0: 3354
mwm_tessrgb_core_0.1.0: 4707
bhm_aqmes_wide3_0.1.0: 1718
bhm_aqmes_wide3-faint_0.1.0: 2550
bhm_aqmes_wide2_0.1.0: 1109
bhm_aqmes_wide2-faint_0.1.0: 736
bhm_aqmes_bonus-dark_0.1.0: 16725
bhm_aqmes_bonus-bright_0.1.0: 1703
bhm_csc_boss-dark_0.1.0: 38684
bhm_csc_boss-bright_0.1.0: 3355
bh

# Flags have attributes

The counts above are for targets assigned to a specific carton, with a specific version. Sometimes we don't care about the carton version and we just want to count by the effective number of targets assigned to that carton (over all versions), or we might want to count by the number of targets assigned to each mapper, or some other attribute.

The `TargetingFlags.mapping` attribute contains a reference dictionary for all cartons, including their attributes. Here's how we can count targets by those attributes:


In [6]:
for mapper, count in flags.count_by_attribute("mapper").items():
    print(f"{mapper}: {count:,}")

na: 0
mwm: 460,377
ops: 325,569
bhm: 65,114
open: 363,471


In [7]:
for program, count in flags.count_by_attribute("program").items():
    print(f"{program}: {count:,}")

na: 0
mwm_snc: 131,249
mwm_cb: 83,708
mwm_halo: 73,865
mwm_yso: 59,667
mwm_rv: 10,209
mwm_ob: 102,320
ops_std: 281,742
mwm_wd: 37,834
mwm_gg: 5,264
mwm_planet: 10,269
bhm_aqmes: 4,337
bhm_filler: 41,024
mwm_tessrgb: 8,065
bhm_csc: 52,291
ops_sky: 0
bhm_rm: 29,196
bhm_spiders: 13,190
mwm_dust: 81
mwm_legacy: 24,789
mwm_galactic: 6,336
mwm_filler: 50,052
mwm_tess_ob: 0
mwm_erosita: 8,323
open_fiber: 363,471
commissioning: 58,684
SKY: 0
mwm_tessob: 0
mwm_magcloud: 0
mwm_bin: 14,175
mwm_validation: 2,413
mwm_monitor: 22


# I own a carton. Where are my spectra?

You can efficiently construct boolean masks to select spectra that are assigned to a carton, program, mapper, or any other attribute.

The easiest way to do this is with the following methods:
- `TargetingFlags.in_carton_pk`: If you know the database primary key (`targetdb.carton.pk`) of your carton
- `TargetingFlags.in_carton_name`: If you know the name of your carton.
- `TargetingFlags.in_carton_label`: If you know the name and version (name_ver) of your carton
- `TargetingFlags.in_mapper`: To select by mapper (e.g., BHM, MWM)
- `TargetingFlags.in_program`: To select by program name.
- `TargetingFlags.in_alt_name`: To select by alternative name, curated with thanks by Marina Koukel.
- `TargetingFlags.in_alt_program`: To select by alternative program name, curated with thanks by Marina Koukel.

If you're not sure what your carton's alternative name or program is, you might find these properties useful:

- `TargetingFlags.all_mappers`: a tuple of all unique mapper names;
- `TargetingFlags.all_programs`: a tuple of all unique program names;
- `TargetingFlags.all_carton_names`: a tuple of all unique carton names;
- `TargetingFlags.all_alt_carton_names`: a tuple of all unique alternative names;
- `TargetingFlags.all_alt_programs`: a tuple of all unique alternative program names;

or you can look this up from `TargetingFlags.mapping`.

In [8]:
print("Mappers:")
for mapper in flags.all_mappers:
    print(f"\t{mapper}")
    
print("Programs")
for program in flags.all_programs:
    print(f"\t{program}")

print("Cartons names")
for name in flags.all_carton_names:
    print(f"\t{name}")

print("Alternative cartons names")
for alt_name in flags.all_alt_carton_names:
    print(f"\t{alt_name}")

print("Alternative programs")
for alt_program in flags.all_alt_programs:
    print(f"\t{alt_program}")


Mappers:
	na
	bhm
	mwm
	ops
	open
Programs
	mwm_filler
	mwm_tessob
	mwm_magcloud
	open_fiber
	mwm_legacy
	SKY
	mwm_validation
	mwm_erosita
	mwm_tessrgb
	na
	bhm_rm
	mwm_halo
	mwm_yso
	mwm_monitor
	commissioning
	mwm_snc
	mwm_tess_ob
	mwm_planet
	mwm_gg
	mwm_bin
	mwm_rv
	bhm_filler
	mwm_cb
	bhm_spiders
	bhm_csc
	mwm_galactic
	ops_std
	bhm_aqmes
	mwm_dust
	mwm_ob
	mwm_wd
	ops_sky
Cartons names
	ops_apogee_stds
	mwm_yso_ob
	bhm_spiders_agn_gaiadr2
	mwm_snc_100pc
	mwm_yso_s2-5
	mwm_yso_pms_apogee_sagitta_edr3_single
	mwm_halo_sm
	mwm_bin_rv_short_mdwarf_apogee_08epoch
	bhm_aqmes_bonus-bright
	mwm_cb_gaiagalex_apogee
	openfibertargets_nov2020_26
	mwm_monitor_n188_apogee_long
	mwm_astar_core_boss
	mwm_yso_cluster_boss
	ops_sky_boss_best
	mwm_erosita_compact_boss
	manual_offset_mwmhalo_offa
	manual_bright_target_offsets_3
	manual_mwm_halo_distant_kgiant_far_boss_single
	mwm_rv_long-fps
	mwm_magcloud_agb_apogee
	openfibertargets_nov2020_17
	ops_sky_boss
	mwm_legacy_ir2opt
	mwm_ob_cepheids_boss

### Select by Mapper
If you want to check which rows are in a particular mapper

In [9]:
# Select things assigned to MWM
mwm_mask = flags.in_mapper("mwm")
mwm_mask

array([ True, False,  True, ..., False,  True, False])

In [10]:
print(f"There are {np.sum(flags.in_mapper('mwm')):,} things assigned to MWM")

There are 460,377 things assigned to MWM


### Select by Program
If you want to check which rows are in a particular program

In [11]:
flags.in_program("mwm_wd")

array([False, False,  True, ..., False, False, False])

### Select by name
If you want to check which rows are in a particular carton

In [12]:
flags.in_carton_name("mwm_snc_250pc")

array([False, False, False, ..., False, False, False])

## I have a target, how was it targeted?

If you have a CATALOGID or SDSS_ID and would like to know the targeting cartons for that source

In [13]:
catalogid = 27021598754184543
idx = np.where((boss_data["CATALOGID21"] == catalogid) |
               (boss_data["CATALOGID25"] == catalogid) | 
               (boss_data["CATALOGID31"] == catalogid))[0]
flag1 = TargetingFlags(boss_data["SDSS5_TARGET_FLAGS"][idx])
for label, count in flag1.count(skip_empty=True).items():
    print(f"{label}")

mwm_yso_s3_0.1.0
mwm_yso_cluster_0.1.0
mwm_yso_variable_apogee_0.5.0
mwm_yso_variable_boss_0.5.0
mwm_yso_cluster_apogee_0.5.0
mwm_yso_cluster_boss_0.5.0
mwm_yso_cluster_apogee_0.5.17
mwm_yso_cluster_boss_0.5.17
mwm_yso_variable_apogee_0.5.17
mwm_yso_variable_boss_0.5.17
mwm_yso_cluster_apogee_1.0.6
mwm_yso_cluster_boss_1.0.6
mwm_yso_variable_apogee_1.0.6
mwm_yso_variable_boss_1.0.6
mwm_yso_variable_apogee_single_1.0.33
mwm_yso_variable_boss_single_1.0.33
mwm_yso_cluster_apogee_single_1.0.33
mwm_yso_cluster_boss_single_1.0.33


In [14]:
sdss_id = 87406300
idx = np.where(boss_data["SDSS_ID"] == sdss_id)[0]
flag1 = TargetingFlags(boss_data["SDSS5_TARGET_FLAGS"][idx])
for label, count in flag1.count(skip_empty=True).items():
    print(f"{label}")

mwm_yso_s3_0.1.0
mwm_yso_cluster_0.1.0
mwm_yso_variable_apogee_0.5.0
mwm_yso_variable_boss_0.5.0
mwm_yso_cluster_apogee_0.5.0
mwm_yso_cluster_boss_0.5.0
mwm_yso_cluster_apogee_0.5.17
mwm_yso_cluster_boss_0.5.17
mwm_yso_variable_apogee_0.5.17
mwm_yso_variable_boss_0.5.17
mwm_yso_cluster_apogee_1.0.6
mwm_yso_cluster_boss_1.0.6
mwm_yso_variable_apogee_1.0.6
mwm_yso_variable_boss_1.0.6
mwm_yso_variable_apogee_single_1.0.33
mwm_yso_variable_boss_single_1.0.33
mwm_yso_cluster_apogee_single_1.0.33
mwm_yso_cluster_boss_single_1.0.33


## About this notebook

This notebook was created to give an example of using the `semaphore` tool with the mwmAllVisit file

**Authors**: Andy Casey (andrew.casey@monash.edu), Sean Morrison

**Last Update**: 2025 Jul 9

If you use `astropy`  for published research, please cite the
authors. Follow this link for more information about citing `astropy`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)

And of course please [cite](https://sdss.org/collaboration/citing-sdss/) SDSS when you use our data.